In [1]:
from sklearn.datasets import load_wine
from datetime import timedelta
import pandas as pd
import numpy as np

In [2]:
read_data_file = pd.read_pickle('../hive.pkl')
df = pd.DataFrame(read_data_file)

In [3]:
df['cal_time'][0:20].median()

Timedelta('15 days 00:43:00')

In [4]:
df['cal_time_binaly'] = df['cal_time'].apply(lambda x:  0 if x < timedelta (days = 15) else 1)

In [5]:
X = df[['commit_befor ', 'commit_after', 'cal_smell', 'dispen_bf', 'object_bf',
       'bloater_bf', 'dispen_at', 'object_at', 'bloater_af', 'open_time',
       'closed_time','changed_file', 'additions', 'deletions',
       'dev']]
y = df['cal_time_binaly']

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test = X[:20], X[20:]
y_train, y_test = y[:20], y[20:]
clf = GradientBoostingClassifier(n_estimators=1, learning_rate=1.0 , max_depth=1 , random_state=2).fit(X_train , y_train)

In [7]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.66666667, 0.66666667, 0.5       , 0.5       , 0.5       ])

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y , y_pred)

array([[ 0, 13],
       [ 0, 17]], dtype=int64)

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred ,average='macro'))
print('recall_score = ', recall_score(y,y_pred ,average='macro'))
print('f1_score',f1_score(y,y_pred ,average='macro'))


precision =  0.2833333333333333
recall_score =  0.5
f1_score 0.36170212765957444


C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.model_selection import GridSearchCV

parameters = {
  
    'learning_rate': [0.01 , 0.1 ,1],
    'n_estimators': [12,14,16],
    'subsample' : [0.7],
    'max_depth' : [2] ,
    'random_state':[2]
  
}
GradientBoosting = GradientBoostingClassifier()
clf = GridSearchCV(GradientBoosting, parameters , cv=3 , scoring='f1_macro')
clf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 1], 'max_depth': [2],
                         'n_estimators': [12, 14, 16], 'random_state': [2],
                         'subsample': [0.7]},
             scoring='f1_macro')

In [12]:
y_pred = cross_val_predict(clf.best_estimator_,X_test,y_test)
y_pred

C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


array([1, 1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=int64)

In [17]:
clf.best_estimator_

GradientBoostingClassifier(learning_rate=1, max_depth=2, n_estimators=12,
                           random_state=2, subsample=0.7)

In [13]:
y_pred_clf = clf.best_estimator_.predict(X_test)

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y_test,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y_test,y_pred_clf ,average='macro'))
print('f1_score',f1_score(y_test,y_pred_clf ,average='macro'))

precision =  0.47619047619047616
recall_score =  0.47619047619047616
f1_score 0.4


In [15]:
import itertools
from itertools import product 

In [16]:
result = itertools.combinations(X ,2)

for item in result:
    df  = pd.DataFrame(item)
    X[list (item)]
    parameters = {
    'learning_rate': [0.01 , 0.1 ,1],
    'n_estimators': [12,14,16],
    'subsample' : [0.7],
    'max_depth' : [2] ,
    'random_state':[2]
    }
    GradientBoosting = GradientBoostingClassifier()
    clf = GridSearchCV(GradientBoosting , parameters , cv=3 , scoring='f1_macro')
    clf.fit(X_train[list (item)],y_train)
    clf.predict(X_test[list (item)])
    #print('predict=',clf.predict(X_test[list (item)]))
    #print(X_test)
    #print("y_train = ",y_train)
    print(item)
    print("precision_score =",precision_score(clf.predict(X[list(item)]),y))
    #print("recall_score =", recall_score(clf.predict(X[list(item)]),y))
    #print("f_1=",f1_score(clf.predict(X[list(item)]),y,average='macro'))
    #print("f_1_X-test=",f1_score(clf.predict(X_test[list(item)]),y_test,average='macro'))

('commit_befor ', 'dispen_bf')
precision_score = 0.7058823529411765
('commit_befor ', 'object_bf')
precision_score = 0.4117647058823529
('commit_befor ', 'bloater_bf')
precision_score = 0.8235294117647058
('commit_befor ', 'open_time')
precision_score = 0.7647058823529411
('dispen_bf', 'object_bf')
precision_score = 0.47058823529411764
('dispen_bf', 'bloater_bf')
precision_score = 0.9411764705882353
('dispen_bf', 'open_time')
precision_score = 0.8235294117647058
('object_bf', 'bloater_bf')
precision_score = 0.4117647058823529
('object_bf', 'open_time')
precision_score = 0.9411764705882353
('bloater_bf', 'open_time')
precision_score = 0.6470588235294118
